# 트리의 앙상블

* 앙상블 모델(ensemble learning) : 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘
* 대부분 결정 트리를 기반으로 만들어져 있음

* 정형 데이터
    * csv, 엑셀, json, xml 등 구조화 된 방식으로 된 데이터
* 비정형 데이터
    * 이미지, 영상, 텍스트, 오디오

### 랜덤포레스트 (random forest)
* 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용됨
* 결정 트리를 랜덤하게 만들어 나무의 숲을 만든다 
* 그리고, 각 결정 트리의 예측을 사용해 최종 예측 

* 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 추출
* 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 생성
* 샘플 중복 가능
* ex) 
    * 1000개 샘플이 담긴 가방에서 100개씩 샘플을 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣는다
    * 이런 식으로 계속해서 100개를 가방에서 뽑으면 중복된 샘플을 뽑을 수 있다
    * 이렇게 만들어진 샘플 : **부트스트랩 샘플(bootstrap sample)** 
    * 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다

* 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다

* **RandomForestClassifier (분류 모델)**
    * 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택 
    * 4개의 특성이 있다면, 노드마다 2개를 랜덤하게 선택하여 사용
* **RandomForestRegressor (회귀 모델)**
    * 전체 특성 사용

* 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 위 방식으로 훈련
    * 분류 : 각 트리의 클래스별 확률을 평균하여, 가장 높은 확률을 가진 클래스를 예측으로 삼는다 
    * 회귀 : 평균을 출력

* 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용 ->
    * 훈련 세트에 과대적합 되는 것을 방지
    * 검증 세트와 테스트 세트에서 안정적인 성능 도출 
* 종종 기본 매개변수 설정만으로도 아주 좋은 결과를 도출 

---

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

* return_train_score 매개변수를 True로 지정 : 검증 점수, 훈련 세트에 대한 점수를 같이 반환 
* 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이함

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

In [12]:
scores

{'fit_time': array([0.4637835 , 0.45580506, 0.464782  , 0.45680141, 0.45779943]),
 'score_time': array([0.05924201, 0.06223321, 0.07021165, 0.06223488, 0.05924082]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [17]:
print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.9973541965122431
test_score : 0.8905151032797809


* 변수 중요도 출력

In [20]:
rf.fit(train_input, train_target)

print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


* 결정 트리의 중요도하고 값이 다른 이유 :
    * 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련시키기 때문 
    * 그 결과, 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다
    * 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다 
    * (TIL_0429_decision_tree.ipynb 참고)

### OOB(Out of Bag)
* OOB 샘플 : 부트스트랩 샘플에 포함되지 않고, 남는 샘플
* OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가 가능 
* oob_score 파라미터의 값을 True로 지정

In [28]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)

print(rf.oob_score_)

0.8934000384837406


---

## 엑스트라트리
* 엑스트라 트리(Extra Trees) : 랜덤 포레스트와 매우 비슷하게 동작
* 기본적으로 100개의 결정 트리를 훈련

* 랜덤 포레스트와 차이점 : 
    * 부트스트랩 샘플을 사용하지 않는다 

* 결정 트리를 만들 때 전체 훈련 세트를 사용 : 
    * 대신 노드를 분할할 때, 가장 좋은 분할을 찾는 것이 아니라 **무작위**로 분할

* DesicionTreeClassifier의 splitter 매개변수 : random으로 지정
    * -> 엑스트라 트리가 사용하는 결정 트리

* 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과를 볼수 있다

In [30]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.9974503966084433
test_score : 0.8887848893166506


In [44]:
et.fit(train_input, train_target)

print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


[알코올 동수, 당도, ph]

* 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에, 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다
* 하지만, 랜덤하게 노드를 분할하기 때문에 **빠른 계산 속도**가 엑스트라 트리의 장점
    * 결정 트리는 최적의 분할을 찾는 데 시간을 많이 소모
    * 특히, 고려해야 할 특성의 개수가 많을 때 더 그렇다
    * 만약, 무작위로 나눈다면 휠씬 빨리 트리를 구성할 수 있다

---

## 그레이디언트 부스팅
* gradient boosting은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방식
* 사이킷런의 GradientBoostingClassifier : 기본적으로 깊이가 3인 결정 트리를 100개 사용

* 깊이가 얕은 결정 트리를 사용 : 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다
* 경사 하강법을 사용합니다
    * 이 방식으로 트리를 앙상블에 추가

* 분류 : **로지스틱 손실 함수** 사용 
* 회귀 : **평균 제곱 오차 함수** 사용

In [47]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.8881086892152563
test_score : 0.8720430147331015


In [48]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.02, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.8872908264055726
test_score : 0.8720428296438882


In [50]:
gb.fit(train_input, train_target)

print(gb.feature_importances_)

[0.12185869 0.74162442 0.13651689]


* 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻는다
* 하지만, 순서대로 트리를 추가하기 때문에 훈련 속도가 **느리다**

---

## 히스토그램 기반 부스팅
* 히스토그램 기반 그레디언트 부스팅(Histogram-based Gradient Boosting) : 정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기 있는 알고리즘

* 먼저 입력 특성을 256개의 구간으로 나눈다
* 따라서, 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다
* max_iter 매개변수 : 부스팅 반복 횟수를 지정

In [52]:
from sklearn.experimental import enable_hist_gradient_boosting

In [55]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.9321723946453317
test_score : 0.8801241948619236


In [56]:
hgb.fit(train_input, train_target)
hgb.score(test_input, test_target)

0.8723076923076923

### Histogram-based Gradient Boosting의 특성 중요도
* permutation_importance()
* 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산
* 훈련 세트 뿐만 아니라 테스트 세트에서도 적용할 수 있음
* 사이킷런에서 제공하는 추정기 모델에 모두 사용 가능
* n_repeats 매개변수 : 랜덤하게 섞을 횟수를 지정

In [64]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state=42, n_jobs=-1)

# train data
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [65]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state=42, n_jobs=-1)

# test data
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


#### XGBoost

conda install -c anaconda py-xgboost

In [67]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.9555033709953124
test_score : 0.8799326275264677


#### LightGBM

conda install -c conda-forge lightgbm

In [68]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print('train_score : {}\ntest_score : {}'.format(np.mean(scores['train_score']), np.mean(scores['test_score'])))

train_score : 0.935828414851749
test_score : 0.8801251203079884
